# Project 3 - What's cooking!

## Setup

### Imports

All the imports go here, to make it easier to find them later.

In [ ]:
import numpy as np
import pandas as pd
import re
import tensorflow as tf
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import random
from sklearn.metrics import r2_score, mean_squared_error, accuracy_score, log_loss

from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

### Reding in the training file

In [ ]:
data = pd.read_json('train.json') 

print("An example of what the data looks like:")
print("")
print(data)

print("")
print("Here the number of recepies of each cuisine")
print("")
print(data['cuisine'].value_counts())

In [ ]:
# The set of different cuisines
cuisines = data.cuisine.unique()

# To find the different ingredients, we need to clean them up a little. 
def clean(string) :
    s = string.replace('-',' ') # read low-fat the same as low fat
    s = string.replace('&', 'and') # read & and and as the same 
    s = re.sub('\((.*?)\)', '', s) # remove everythin g in brackets
    s = re.sub('\d{1,2}\%', '', s) # remove things of the form d% or dd%, where d is a digit
    s = ' '.join(s.split()) # remove extra white spaces
    
    return s

ing_list = data.ingredients.values.tolist()
raw_ingredients = [clean(x) for ing in ing_list for x in ing]

ingredients = sorted(set(raw_ingredients))

print("There are %d different ingredients." % len(ingredients))
print("")
print("")
print("Here is the very long list:")
print("")
print(ingredients)
    

### Recipies as vectors

Here we have functions that take a list of incredients and produces a vector to represent them. 


In [ ]:
# build a dictionary that to each ingredient assigns its index
ingredient_index = {}
for i in range(0,len(ingredients)) :
    ingredient_index[ingredients[i]] = i

# the same for cuisines 
cuisine_index = {}
for i in range(0, len(cuisines)) : 
    cuisine_index[cuisines[i]] = i 
    
def ingredients_to_vector(ings) :
    vect = np.zeros(len(ingredients))
    for ing in ings :
        vect[ingredient_index[clean(ing)]] = 1
        
    return vect

def cuisine_to_vector(cus) : 
    vect = np.zeros(20)
    vect[cuisine_index[cus]] = 1
    return vect

vect_list = [ingredients_to_vector(ing) for ing in ing_list]
target_list = [cuisine_to_vector(cus) for cus in data.cuisine.values.tolist()]

print(len(vect_list))
print(len(target_list))

print(vect_list[30064])
print(target_list[30064])


In [ ]:
for t in random.sample(target_list, 20) :
    print(t)
    
print(cuisine_index)

## Split data

In [ ]:
X = np.c_[vect_list]
Y = np.c_[target_list]

print(X.shape)
print(Y.shape)

x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size = 0.2)

print('Shape of x_train: '+ str(x_train.shape))
print('Shape of y_train: '+ str(y_train.shape))
print()
print('Shape of x_test: '+ str(x_test.shape))
print('Shape of y_test: '+ str(y_test.shape))

## Alter the target data to output numbers instead of arrays

In [ ]:
Y_num = np.zeros((Y.shape[0]))
for i in range(Y.shape[0]):
    Y_num[i] = np.argmax(Y[i])

print(Y_num)
    
x_train, x_test, y_train, y_test = train_test_split(X, Y_num, test_size = 0.2)

## Random Forests
Let's make some forests with different sizes!


In [ ]:
print('Starting training:')

for i in [1, 10, 20, 50, 100]:
    clf = RandomForestClassifier(n_estimators=i, max_depth=None, max_features=None,
                             verbose=True, n_jobs=8)
    clf.fit(x_train, y_train)
    print()
    print('Some guesses: ' + str(clf.predict(x_test[90:95])))
    print('Actual answers: ' + str(y_test[90:95]))
    
    print('Accuracy on test data when we have %s tree(s):' %i)
    print(clf.score(x_test, y_test))
    

## Predicting with more trees forest

In [23]:
print('Starting training:')

for i in [100, 150, 200, 250]:
    clf = RandomForestClassifier(n_estimators=i, max_depth=None, max_features=None,
                             verbose=True, n_jobs=8)
    clf.fit(x_train, y_train)
    print()
    print('Some guesses: ' + str(clf.predict(x_test[90:95])))
    print('Actual answers: ' + str(y_test[90:95]))
    
    print('Accuracy on test data when we have %s tree(s):' %i)
    print(clf.score(x_test, y_test))

Starting training:


[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:  9.1min
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed: 21.8min finished


[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    0.4s finished


Some guesses: [ 6. 10.  1.  1.  3.]
Actual answers: [ 6.  3. 12. 14.  3.]
Accuracy on test data when we have 100 tree(s):


[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    0.2s finished


...

[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:  8.0min
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed: 39.0min
[Parallel(n_jobs=8)]: Done 250 out of 250 | elapsed: 52.7min finished


[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 250 out of 250 | elapsed:    0.0s finished


Some guesses: [ 6. 10.  1.  1.  3.]
Actual answers: [ 6.  3. 12. 14.  3.]
Accuracy on test data when we have 250 tree(s):


[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.5s
[Parallel(n_jobs=8)]: Done 250 out of 250 | elapsed:    0.6s finished


0.6736643620364551


## Now with the square root as n_features

In [24]:
print('Starting training:')

for i in [100, 150, 200, 250]:
    clf = RandomForestClassifier(n_estimators=i, max_depth=None,
                             verbose=True, n_jobs=8)
    clf.fit(x_train, y_train)
    print()
    print('Some guesses: ' + str(clf.predict(x_test[90:95])))
    print('Actual answers: ' + str(y_test[90:95]))
    
    print('Accuracy on test data when we have %s tree(s):' %i)
    print(clf.score(x_test, y_test))

Starting training:


[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:   14.6s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:   37.6s finished



Some guesses: [ 6. 10.  1.  1.  3.]
Actual answers: [ 6.  3. 12. 14.  3.]
Accuracy on test data when we have 100 tree(s):


[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    0.2s finished


0.7088623507228159


...

0.7145191703331238


[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:   14.5s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:  1.2min
[Parallel(n_jobs=8)]: Done 250 out of 250 | elapsed:  1.6min finished



Some guesses: [6. 3. 1. 1. 3.]
Actual answers: [ 6.  3. 12. 14.  3.]
Accuracy on test data when we have 250 tree(s):


[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 250 out of 250 | elapsed:    0.0s finished
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.5s
[Parallel(n_jobs=8)]: Done 250 out of 250 | elapsed:    0.8s finished


0.7133878064110623
